In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from ml_lib.models.pla import PerceptronLearningAlgorithm
%matplotlib qt
rng = np.random.default_rng(seed=21)  # Set the seed for the random generator

In [2]:

fig_size = (14, 10)
RED = "#E57873" #Red
BLUE = "#94CAE0" #Blue
ORANGE = "#FF9D26" #Orange
PRETTY_CMAP = ListedColormap([BLUE, RED, ORANGE])

def get_spiral_data(num_points=100, num_classes=3, num_features=2, save=False):
    N = num_points # number of points per class
    K = num_classes # number of classes
    D = num_features  # dimensionality

    X = np.zeros((N*K, D)) # data matrix (each row = single example)
    y = np.zeros(N*K, dtype='uint8') # class labels
    for j in range(K):
        ix = range(N*j,N*(j+1))
        r = np.linspace(0.0,1,N) # radius
        t = np.linspace(j*4,(j+1)*4, N) + rng.integers(0, N)*0.2 # theta
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        y[ix] = j
        
    # lets visualize the data:
    plt.figure(figsize=fig_size)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=80, cmap=PRETTY_CMAP, edgecolors='k')
    # equal axis
    plt.axis('equal')
    plt.title('Spiral Data')
    plt.show()
    return X, y

# Generate a 2D classification dataset perfectly seperated by a line
def get_linear_data(n_points=100, save_fig = False, margin=1.0):
    '''
    Generate a 2D classification dataset perfectly seperated by a line

    :param n_points: the number of data points
    :param save_fig: whether to save the figure
    :param margin: the margin of the classifier
    :return x: x values [n_points x 2]
    :return y: y values [n_points x 1]
    :return w: the normal vector of the line
    :return b: the bias term of the line
    '''
    # Generate n_points between [-10, 10]
    data_range = np.array([-10, 10])

    # Generate a random line y = mx + c where m is the slope and c is the y-intercept
    w = rng.random(2) # Randomly generate the slope and y-intercept
    b = rng.random(1)
    
    i_points = 0
    x = np.empty((n_points, 2))
    y = np.empty((n_points, 1))

    while(i_points < n_points):
        x1 = rng.uniform(*data_range)
        x2 = rng.uniform(*data_range)
        x[i_points] = [x1, x2]

        # Assign labels based on the line if they perfectly lie on one side of the line
        distance = distance_from_line((x1, x2), w, b)
        if abs(distance) < margin:
            continue
        else:
            y_i = np.sign(w.dot(x[i_points].T)-b)
            y[i_points] = y_i
            i_points += 1
    return x, y, w, b

def plot_data(x, y, fig = None):
    # Plot the data
    if fig is None:
        fig = plt.figure(figsize=fig_size)
    else:
        plt.figure(fig.number)
    
    # Label both the data points and the line
    labels = ['Class 1', 'Class 2', 'Line']
    plt.scatter(x[:, 0], x[:, 1], c=y, s=100, cmap=PRETTY_CMAP, edgecolors='k')
    plt.legend(labels)
    plt.axis('equal')
    plt.title('Linearly Separable Data')
    return fig

def plot_decision_boundary(x, y, w, b, label=None, fig = None, alpha=0.5, label_line=True):

    if fig is None:
        fig = plt.figure(figsize=fig_size)
    else:
        plt.figure(fig.number)

    # Calculate the decision boundary
    x1 = x[:, 0]
    x2 = x[:, 1]
    x1_boundary = x1
    x2_boundary = (-w[0]*x1_boundary + b)/w[1]
    
    # Calculate the center of the line
    x1_center = (np.max(x1_boundary) + np.min(x1_boundary))/2
    x2_center = (np.max(x2_boundary) + np.min(x2_boundary))/2
    
    # normalize the normal vector
    w_normed = 5*w/np.linalg.norm(w)

    # Plot the decision boundary and the normal vector    
    c = RED
    if label == 'True Decision Boundary':
        c = BLUE

    plt.plot(x1_boundary, x2_boundary, c=c, label=label, linewidth=2, linestyle='--', alpha=alpha)
    plt.quiver(x1_center, x2_center, w_normed[0], w_normed[1], angles='xy', scale_units='xy', scale=1, color=c, alpha=alpha)

    # Calculate the margin of the classifier
    min_margin, _ = compute_margin(x, y, w, b)
    mistake_bound_val = mistake_bound(x, min_margin)
    misclassifications = misclassified_points(x, y, w, b)
    plt.legend()
    plt.title('Linearly Separable Data with Margin: {:.2f}, Mistake Bound: {:.2f}, Misclassifications: {:.1f}'.format(min_margin, mistake_bound_val, misclassifications))
    plt.axis('equal')
    
    xlim = np.array([np.min(x1), np.max(x1)])
    ylim = np.array([np.min(x2), np.max(x2)])

    plt.xlim(1.1*xlim)
    plt.ylim(1.1*ylim)
    return fig

def distance_from_line(x, w, b):
    """
    Compute the distance of a point from a line
    x: a point
    w: the normal vector of the line
    b: the bias term of the line
    """
    distance = (w.T.dot(x) - b)/np.linalg.norm(w)
    return distance

def misclassified_points(x, y, w, b):
    """
    Compute the number of misclassified points
    x: the data points
    y: the labels
    w: the normal vector of the line
    b: the bias term of the line
    """
    misclassified_points = 0
    for i in range(x.shape[0]):
        if y[i] != np.sign(np.dot(x[i], w) - b):
            misclassified_points += 1
    return misclassified_points

# Margin of the classifier is calculated as the maximum distance of the misclassified points from the line 
def compute_margin(x, y, w, b):
    """
    Compute the margin of a point from a line
    x: a point
    w: the normal vector of the line
    b: the bias term of the line
    """
    margin = y.T*(w.dot(x.T))/np.linalg.norm(w)

    # Find the minimum margin and the point that has the minimum margin
    min_margin = np.min(margin)
    min_margin_point = x[np.argmin(margin)]
    return min_margin, min_margin_point

def mistake_bound(x, margin):
    """
    Maximum number of mistakes perceptron algorithm could make on a linearly separable dataset
    https://svivek.com/teaching/lectures/slides/perceptron/perceptron-mistake-bound.pdf
    n_features: the number of features
    margin: the margin of the classifier
    """
    R = np.max(np.linalg.norm(x, axis=1))
    return (R**2)/(margin**2)

In [3]:
# Get the data and the true line
x, y, w, b = get_linear_data(margin=1.0)

# Plot the data
fig = plot_data(x, y)

# Initialize the Perceptron Learning Algorithm
pla = PerceptronLearningAlgorithm(learning_rate=0.01, rng=rng)

# set misclassified point to inf
misclassified = misclassified_points(x, y, pla.weights, pla.bias)
last_w = pla.weights
last_b = pla.bias
iteration = 0
while misclassified > 0:
    # Clear the display and plot the data and the true line
    plt.clf()
    plot_data(x, y, fig)
    plot_decision_boundary(x, y, w, b, 'True Decision Boundary', fig)
    
    # Get random data point
    rnd_idx = rng.integers(0, x.shape[0])
    x_i = x[rnd_idx].reshape(1, 2)
    y_i = y[rnd_idx]
    
    # Perform prediction on the current to get the current decision boundary
    y_pred_i = pla.predict(x_i)
    
    # Mark the point that was used for training based on the prediction
    edge_color = 'g'
    if y_pred_i != y_i:
        edge_color = 'r'

    # Plot the last decision boundary    
    last_w = pla.weights
    last_b = pla.bias
    plot_decision_boundary(x, y, last_w, last_b, 'Last Decision Boundary', fig=fig, alpha=0.2)

    # Mark the point that was used for training
    plt.scatter(x_i[0, 0], x_i[0, 1], s=140, edgecolors=edge_color, facecolors='none', linewidth=3, label='Training Point')

    # Update the model based on the current point.
    # Model is updated only if the prediction is incorrect
    pla.train(x_i, y_i)

    # Plot the new decision boundary
    plot_decision_boundary(x, y, pla.weights, pla.bias, 'Predicted Decision Boundary', fig=fig, alpha=0.75)
    
    # Calculate the margin of the classifier
    # y_pred = pla.predict(x)
    margin, min_margin_pt = compute_margin(x, y, pla.weights, pla.bias)

    # Mark the point that has the minimum margin
    plt.scatter(min_margin_pt[0], min_margin_pt[1], s=140, edgecolors='b', facecolors='none', linewidth=3, label='Margin Point')
    plt.legend()
    
    # get parent working directory
    pwd = os.path.dirname(os.getcwd())

    # get the repository path not the cwd
    result_path = pwd + "/results/bias_variance_tradeoff/"
    # file = str(iteration) + 'pla.png'
    # iteration += 1
    # plt.savefig(file, dpi=600)
    
    # Calculate the number of misclassified points
    misclassified = misclassified_points(x, y, pla.weights, pla.bias)
    plt.pause(0.001)
    plt.waitforbuttonpress()


Number of misclassified points:  15  Margin:  -4.549562690205908
Number of misclassified points:  15  Margin:  -4.549562690205908
Number of misclassified points:  15  Margin:  -4.549562690205908
Number of misclassified points:  15  Margin:  -4.549562690205908
Updated weights: [0.33316542 0.47495883], bias: [0.56458904]
Number of misclassified points:  10  Margin:  -3.051705530944294
Number of misclassified points:  10  Margin:  -3.051705530944294
Number of misclassified points:  10  Margin:  -3.051705530944294
Number of misclassified points:  10  Margin:  -3.051705530944294
Number of misclassified points:  10  Margin:  -3.051705530944294
Number of misclassified points:  10  Margin:  -3.051705530944294
Updated weights: [0.36438289 0.46301921], bias: [0.55458904]
Number of misclassified points:  8  Margin:  -2.4084170408166177
Number of misclassified points:  8  Margin:  -2.4084170408166177
Number of misclassified points:  8  Margin:  -2.4084170408166177
Number of misclassified points:  